https://quantumcomputing.stackexchange.com/questions/6236/how-to-quickly-calculate-the-custom-u3-gate-parameters-theta-phi-and-lamb
https://quantumcomputing.stackexchange.com/questions/11410/lambda-parameter-for-u3-gate-in-qiskit-bloch-sphere-visualisation
https://quantumcomputing.stackexchange.com/questions/7147/why-use-u2-and-u1-gate-in-ibm-quantum-computers

In [1]:
import qiskit
import numpy as np

In [2]:
# Pauli matricies
sigma = {}
sigma_x = np.matrix([[0, 1], [1, 0]])
sigma_y = np.matrix([[0, -1j], [1j, 0]])
sigma_z = np.matrix([[1, 0], [0, -1]])

# rotation about an arbitrary axis
def rot(theta, nvec):
    rot_matrix = nvec[0]*sigma_x + nvec[1]*sigma_y + nvec[2]*sigma_z
    return np.cos(theta)*np.identity(2) + 1j*np.sin(theta)*rot_matrix

# rotation about x, y, z[]
def rx(theta):
    return rot(-0.5*theta, [1, 0, 0])

def ry(theta):
    return rot(-0.5*theta, [0, 1, 0])

def rz(theta):
    return np.exp(1j*(theta)/2) * rot(-0.5*theta, [0, 0, 1])

# U3 rotation
def rot_u3(theta, phi, lam):
    return rz(phi) * ry(theta) * rz(lam)

# get unitary for single qubit gates from qiskit unitary simulator
def qiskit_get_unitary(op, *args, **kwargs):
    qc = qiskit.QuantumCircuit(1)
    getattr(qc, op)(*args, 0, **kwargs)
    backend = qiskit.BasicAer.get_backend('unitary_simulator')
    job = qiskit.execute(qc, backend)
    return job.result().get_unitary()

# U3 rotation by qiskit
def qiskit_rot_u3(theta, phi, lam):
    return qiskit_get_unitary('u3', theta, phi, lam)

# rotation about x, y, z by qiskit
def qiskit_rx(theta):
    return qiskit_get_unitary('rx', theta)

def qiskit_ry(theta):
    return qiskit_get_unitary('ry', theta)

def qiskit_rz(theta):
    return qiskit_get_unitary('rz', theta)

print("------- U3 rotation -------")
print("calculated:")
print(rot_u3(np.pi, np.pi, np.pi))
print("from qiskit:")
print(qiskit_rot_u3(np.pi, np.pi, np.pi))

print("------- Ry rotation -------")
print("calculated:")
print(ry(np.pi))
print("from qiskit:")
print(qiskit_ry(np.pi))

------- U3 rotation -------
calculated:
[[ 6.123234e-17+0.00000000e+00j  1.000000e+00-1.22464680e-16j]
 [-1.000000e+00+1.22464680e-16j  6.123234e-17-1.49975978e-32j]]
from qiskit:
[[ 0.+0.0000000e+00j  1.-1.2246468e-16j]
 [-1.+1.2246468e-16j  0.+0.0000000e+00j]]
------- Ry rotation -------
calculated:
[[ 6.123234e-17+0.j -1.000000e+00+0.j]
 [ 1.000000e+00+0.j  6.123234e-17+0.j]]
from qiskit:
[[ 0.+0.j -1.+0.j]
 [ 1.+0.j  0.+0.j]]


In [3]:
# rotation matrix to Euler angles (U3)
rot_matrix = rot_u3(np.pi, np.pi, np.pi)
theta, phi, lam, phase = qiskit.quantum_info.synthesis.OneQubitEulerDecomposer(basis='U3').angles_and_phase(rot_matrix)
print("rotation matrix:")
print(rot_matrix)
print("Euler angles:")
print("theta = %s, phi = %s, lambda = %s, phase = %s" % (theta, phi, lam, phase))

rotation matrix:
[[ 6.123234e-17+0.00000000e+00j  1.000000e+00-1.22464680e-16j]
 [-1.000000e+00+1.22464680e-16j  6.123234e-17-1.49975978e-32j]]
Euler angles:
theta = 3.141592653589793, phi = 3.141592653589793, lambda = -3.141592653589793, phase = -1.2246467991473532e-16
